# Importing necessary libraries

In [5]:
import pandas as pd 

# Load all the Files

In [84]:
prices = pd.read_csv("prices.csv") 
auditors = pd.read_csv("auditors.csv") 
stores = pd.read_json('stores.json')

In [85]:
prices.head() # printing the first five rows

,Auditor ID,Date,Price,Store ID,UPC
0,234,10/18/17,24.95,66999,268588472
1,234,10/27/17,49.71,66999,475245085
2,234,10/20/17,25.75,66999,126967843
3,234,10/23/17,18.81,66999,708930835
4,234,10/23/17,33.32,66999,325885139


In [120]:
stores.head()

,Banner,Region,Store ID
0,Walmart,Northern California,66999
1,Trader Joes,Northern California,4698
2,Safeway,Northern California,39482
3,Whole Foods,Northern California,34957
4,Walmart,New York,12837


In [124]:
auditors.head()

,Auditor ID,First,Last,Region
0,234,Sue,Smith,Northern California
1,536,Bob,Smith,Northern California
2,98,Jack,Smith,New York
3,203,Jill,Smith,New York
4,304,Jerry,Johnson,Texas
5,63,Randy,Johnson,Texas
6,1326,Mike,Johnson,Kansas
7,713,Dave,Johnson,Kansas


# The reason behind this join: we need neccessary column data from two tables
We want, 'UPC', 'Price', 'Auditor ID' from Prices table 

And We need, 'Banner' and 'Region' from Stores table

So, we outer joined on 'Store ID'

In [88]:
prices_stores_joined = pd.merge(prices, stores, on = 'Store ID', how = 'outer')
prices_stores_joined.head()

,Auditor ID,Date,Price,Store ID,UPC,Banner,Region
0,234.0,10/18/17,24.95,66999,268588472.0,Walmart,Northern California
1,234.0,10/27/17,49.71,66999,475245085.0,Walmart,Northern California
2,234.0,10/20/17,25.75,66999,126967843.0,Walmart,Northern California
3,234.0,10/23/17,18.81,66999,708930835.0,Walmart,Northern California
4,234.0,10/23/17,33.32,66999,325885139.0,Walmart,Northern California


# The reason behind this join: we only want to count those regions where we have sent auditors

So, we outer joined on 'Author ID'

In [110]:
refined_data_by_auditors = pd.merge(prices_stores_joined, auditors, on = ['Auditor ID'], how = 'outer')
refined_data_by_auditors.head()

,Auditor ID,Date,Price,Store ID,UPC,Banner,Region_x,First,Last,Region_y
0,234,10/18/17,24.95,66999,268588472.0,Walmart,Northern California,Sue,Smith,Northern California
1,234,10/27/17,49.71,66999,475245085.0,Walmart,Northern California,Sue,Smith,Northern California
2,234,10/20/17,25.75,66999,126967843.0,Walmart,Northern California,Sue,Smith,Northern California
3,234,10/23/17,18.81,66999,708930835.0,Walmart,Northern California,Sue,Smith,Northern California
4,234,10/23/17,33.32,66999,325885139.0,Walmart,Northern California,Sue,Smith,Northern California


In [112]:
final_refined_data = refined_data_by_auditors.drop(['Date', 'First','Last','Auditor ID','Region_x'], axis=1) # dropping some unnecessary columns
final_refined_data = final_refined_data.rename(columns = {"Region_y": "Region"})
final_refined_data.head()

,Price,Store ID,UPC,Banner,Region
0,24.95,66999,268588472.0,Walmart,Northern California
1,49.71,66999,475245085.0,Walmart,Northern California
2,25.75,66999,126967843.0,Walmart,Northern California
3,18.81,66999,708930835.0,Walmart,Northern California
4,33.32,66999,325885139.0,Walmart,Northern California


# Generate Output DataFrame

As we want to see product's price or UPC region wise, we make all the regions our column value.

Each column regionwise contains the price of product

In [115]:
output = pd.crosstab(index=[final_refined_data.Banner, final_refined_data.UPC], columns=final_refined_data.Region, margins=True, values=final_refined_data.Price,dropna = False,aggfunc='mean').round(2)
output = output.drop(['All'], axis=1) # Dropping the unnecessary 'All' column

In [116]:
output.head()

Region             Kansas  New York  Northern California  Texas
Banner UPC                                                     
Kroger 11873171.0     NaN       NaN                  NaN    NaN
       15052612.0     NaN       NaN                  NaN    NaN
       16482322.0     NaN       NaN                  NaN    NaN
       16729338.0     NaN       NaN                  NaN    NaN
       16829288.0     NaN       NaN                  NaN    NaN

# Creating a csv file as a result for the output

In [117]:
output.to_csv("Output.csv", index = True)

# For the second part of the question

1. There was no auditor sent to 'Hawai'. So, we dont have any data of 'Hawai'
2. Also, there are products in 'Kroger' but nothing has been logged.